In [4]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)

from sklearn.preprocessing import MinMaxScaler
import joblib

import sys
sys.path.insert(0, "../")
import functions

# <font size="7">Feature Selection</font>
<font size="5">Die Feature Selection erfolgt analog zum Prozess für die Trainingsdaten.</font>

In [5]:
#Daten aus CSV-Datei laden
df = pd.read_csv("../2-Data Understanding/Datenbeschaffung/verbrauch.csv", index_col=0, usecols=[0, 1, 5], parse_dates=True)
df.index.freq = "D"

#Temperatur hinzufügen
df_ = pd.read_csv("../2-Data Understanding/Datenbeschaffung/kalender.csv", index_col=0, parse_dates=True, usecols=[0,2])
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/stuttgart_vorhersage.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("stuttgart"))
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/freiburg_vorhersage.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("freiburg"))
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/mannheim_vorhersage.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("mannheim"))
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/ulm_vorhersage.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("ulm"))

df["temperatur"] = round(((df_["stuttgart"] + df_["freiburg"] + df_["mannheim"] + df_["ulm"]) / 4), 1)

#Sonnenauf-/Sonnenuntergang für Tagesstunden hinzufügen
df = df["2021-08-01":].join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/stuttgart.csv", index_col=0, parse_dates=True, usecols=[0, 8, 9]))

#DataFrame ausgeben
print(df.head())
print()
print(df.describe().transpose())

            verbrauch  arbeitstag  temperatur sonnenaufgang sonnenuntergang
datum                                                                      
2021-08-01     128451           0        15.0      04:57 AM        08:01 PM
2021-08-02     169754           1        16.1      04:58 AM        08:00 PM
2021-08-03     174071           1        15.2      05:00 AM        07:58 PM
2021-08-04     171961           1        14.0      05:01 AM        07:57 PM
2021-08-05     169184           1        14.3      05:02 AM        07:55 PM

            count           mean           std       min       25%       50%       75%       max
verbrauch   153.0  181321.418301  27080.794443  119860.0  161508.0  187846.0  202666.0  231130.0
arbeitstag  153.0       0.712418      0.454122       0.0       0.0       1.0       1.0       1.0
temperatur  153.0      10.216993      6.144604      -2.0       5.5       9.9      15.3      23.8


# <font size="7">Feature Engineering</font>
<font size="5">Das Feature Engineering erfolgt analog zum Prozess für die Trainingsdaten.</font>

In [6]:
#Merkmal für Tagesstunden erzeugen

#Tagesstunden berechnen
df["tagesstunden"] = round((pd.to_timedelta(
    pd.to_datetime(df["sonnenuntergang"]).dt.strftime("%H:%M:%S")).dt.total_seconds() - 
    pd.to_timedelta(pd.to_datetime(df["sonnenaufgang"]).dt.strftime("%H:%M:%S")).dt.total_seconds()) / 3600, 1)

#Sonnenaufgang und Sonnenuntergang werden durch Tagesstunden ersetzt
df.drop(["sonnenaufgang", "sonnenuntergang"], axis=1, inplace=True)

#DataFrame ausgeben
print(df.head())
print()
print(df.describe().transpose())

            verbrauch  arbeitstag  temperatur  tagesstunden
datum                                                      
2021-08-01     128451           0        15.0          15.1
2021-08-02     169754           1        16.1          15.0
2021-08-03     174071           1        15.2          15.0
2021-08-04     171961           1        14.0          14.9
2021-08-05     169184           1        14.3          14.9

              count           mean           std       min       25%       50%       75%       max
verbrauch     153.0  181321.418301  27080.794443  119860.0  161508.0  187846.0  202666.0  231130.0
arbeitstag    153.0       0.712418      0.454122       0.0       0.0       1.0       1.0       1.0
temperatur    153.0      10.216993      6.144604      -2.0       5.5       9.9      15.3      23.8
tagesstunden  153.0      11.044444      2.228313       8.3       8.9      10.8      13.0      15.1


# <font size="7">Featurization</font>
<font size="5">Die Featurization erfolgt analog zum Prozess für die Trainingsdaten.</font>

In [13]:
#Alle Daten speichern
df.to_csv("wettervorhersage.csv")

#Merkmale sklaieren
exog = df[["arbeitstag", "temperatur", "tagesstunden"]]

scaler_exog = joblib.load("scaler_exog.save")

scaled_exog = pd.DataFrame(scaler_exog.transform(exog), columns=exog.columns, index=exog.index)

#Ziel skalieren
endog = df["verbrauch"]

scaler_endog = joblib.load("scaler_endog.save")

scaled_endog = pd.DataFrame(scaler_endog.transform(endog.values.reshape(-1, 1)), columns=["verbrauch"], index=endog.index)

#Ziel und Merkmale zusammenführen
data_scaled = scaled_endog.join(scaled_exog)
data_scaled.to_csv("wettervorhersage_scaled.csv")